In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fim import apriori
from gsp import *

In [17]:
store -r not_canceled

In [18]:
sequential_df = not_canceled.groupby(['CustomerID','BasketID'])['ProdID'].apply(list)

In [19]:
sequential_df.head()

CustomerID  BasketID
12346.0     541431                                                [23166]
12347.0     537626      [85116, 22375, 71477, 22492, 22771, 22772, 227...
            542237      [84625A, 84625C, 85116, 20719, 22375, 22376, 2...
            549222      [22376, 22374, 22371, 22375, 20665, 23076, 217...
            556201      [23084, 23162, 23171, 23172, 23170, 23173, 231...
Name: ProdID, dtype: object

In [20]:
sequential_df = sequential_df.groupby(level=0).apply(list)

In [21]:
sequential_df

CustomerID
12346.0                                            [[23166]]
12347.0    [[85116, 22375, 71477, 22492, 22771, 22772, 22...
12348.0    [[84992, 22951, 84991, 84991, 21213, 21213, 22...
12349.0    [[23112, 23460, 21564, 21411, 21563, 22131, 22...
12350.0    [[21908, 22412, 79066K, 79191C, 22348, 84086C,...
                                 ...                        
18280.0    [[82484, 22180, 22467, 22725, 22727, 22495, 22...
18281.0    [[22037, 22716, 22028, 23007, 23008, 23209, 22...
18282.0    [[21270, 23187, 23295, 22089, 21108, 21109, 22...
18283.0    [[22356, 20726, 22384, 22386, 20717, 20718, 85...
18287.0    [[22755, 22754, 22753, 22756, 22758, 22757, 22...
Name: ProdID, Length: 4334, dtype: object

In [ ]:
#signature requires: the dataset, the min support and the boolean verbose
result_set = apriori(sequential_df, 60, verbose=False)